# Agno Search Systems with DuckDuckGo

Ноутбук демонстрирует три подхода к поиску информации с помощью Agno и DuckDuckGo:

1. **Workflow** — пошаговый флоу на чистых функциях.
2. **Advanced Agent** — один агент с planning / memory / knowledge.
3. **Multi-Agent System** — несколько ролей + Structured Output.

Перед запуском убедитесь, что в `.env` заданы `OPENAI_API_KEY` и `OPENAI_BASE_URL`.


In [24]:
# Импорты и настройка окружения
import dotenv
from rich.console import Console
from rich.panel import Panel
from rich.markdown import Markdown
from typing import List

from pydantic import BaseModel, Field

from agno.agent import Agent
from agno.workflow import Workflow
from agno.models.openai import OpenAIChat
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.memory import MemoryManager

from settings import OpenAISettings

# Подгружаем .env по аналогии с ask_model.ipynb
dotenv.load_dotenv(dotenv.find_dotenv())

console = Console()
settings = OpenAISettings()
api_key = settings.openai_api_key.get_secret_value()
base_url = str(settings.openai_base_url)

console.print("[green]✓ Настройки загружены успешно![/green]")


✓ Настройки загружены успешно!

---
## 1. Workflow с DuckDuckGo

Пошаговый Workflow использует функцию-поисковик и функцию-аналитик. Все вызовы делают DuckDuckGo через Agno tools.


In [25]:
# Создание агентов для workflow
searcher = Agent(
    name="Searcher",
    role="system",
    model=OpenAIChat(
        id="Qwen/Qwen3-Next-80B-A3B-Instruct",
        api_key=api_key,
        base_url=base_url
    ),
    tools=[DuckDuckGoTools()],
    system_message=(
        "Ты специалист по поиску информации.\n"
        "- Используй DuckDuckGo для всех запросов.\n"
        "- Собирай только релевантные факты и указывай источники."
    ),
    markdown=True
)

analyzer = Agent(
    name="Analyzer",
    role="system",
    model=OpenAIChat(
        id="Qwen/Qwen3-Next-80B-A3B-Instruct",
        api_key=api_key,
        base_url=base_url
    ),
    system_message=(
        "Ты аналитик.\n"
        "- Своди результаты поиска в четкое резюме.\n"
        "- Выделяй ключевые выводы и рекомендации."
    ),
    markdown=True
)

# Определение функций-шагов для workflow
def search_step(workflow, step_input):
    """Шаг 1: Поиск информации"""
    query = step_input.input
    result = searcher.run(query)
    return {"search_results": result.content}

def analyze_step(workflow, step_input):
    """Шаг 2: Анализ результатов"""
    search_results = step_input.session_state.get("search_results", "")
    prompt = f"Проанализируй следующие результаты поиска и создай краткое резюме:\n\n{search_results}"
    result = analyzer.run(prompt)
    return {"analysis": result.content}

# Создание workflow
search_workflow = Workflow(
    name="Search and Analyze Workflow",
    steps=[search_step, analyze_step]
)

console.print(Panel(
    "[green]✓ Workflow создан успешно![/green]\n\n"
    "Шаги:\n"
    "1. 🔍 Поиск информации (Searcher)\n"
    "2. 📊 Анализ результатов (Analyzer)",
    title="Search Workflow",
    border_style="blue"
))


╭──────────────────────────────────────────────── Search Workflow ────────────────────────────────────────────────╮
│ ✓ Workflow создан успешно!                                                                                      │
│                                                                                                                 │
│ Шаги:                                                                                                           │
│ 1. 🔍 Поиск информации (Searcher)                                                                               │
│ 2. 📊 Анализ результатов (Analyzer)                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [26]:
# Функция для запуска workflow
def run_workflow_search(query: str):
    """Запускает workflow поиска и анализа"""
    console.print(Panel(
        f"[cyan]Запрос:[/cyan] {query}",
        title="🚀 Запуск Workflow",
        border_style="cyan"
    ))
    
    result = search_workflow.run(query)
    
    if hasattr(result, "content"):
        console.print(Panel(
            Markdown(result.content),
            title="📄 Результат Workflow",
            border_style="green"
        ))
    
    return result

# Пример использования
# run_workflow_search("Какие новые проекты в области искусственного интеллекта?")
console.print("[yellow]Workflow готов! Раскомментируйте строку выше для теста.[/yellow]")


Workflow готов! Раскомментируйте строку выше для теста.

---
## 2. Advanced Agent (Planning + Memory + Knowledge)

Один агент, который строит план, использует DuckDuckGo tools, хранит контекст через `MemoryManager` и подмешивает знания из истории.


In [27]:
# Создание продвинутого агента
advanced_agent = Agent(
    name="Advanced Search Agent",
    role="system",
    model=OpenAIChat(
        id="Qwen/Qwen3-Next-80B-A3B-Instruct",
        api_key=api_key,
        base_url=base_url
    ),
    tools=[DuckDuckGoTools()],
    memory_manager=MemoryManager(),
    enable_agentic_memory=True,
    add_history_to_context=True,
    num_history_runs=5,
    add_knowledge_to_context=True,
    search_knowledge=True,
    enable_agentic_context=True,
    system_message=(
        "Ты продвинутый поисковый агент.\n"
        "1. Всегда используй DuckDuckGo для актуальной информации.\n"
        "2. Для новостей применяй формулировки latest/breaking/recent news.\n"
        "3. Для науки добавляй research, study, paper, journal.\n"
        "4. Для гайдов используй tutorial, guide, how-to, documentation.\n"
        "5. Перед ответом строи план шагов и учитывай историю пользователя.\n"
        "6. Форматируй вывод в Markdown и указывай источники."
    ),
    markdown=True
)

console.print(Panel(
    "[green]✓ Продвинутый агент создан![/green]\n\n"
    "Возможности:\n"
    "• 🧠 Memory: учитывает последние 5 запросов\n"
    "• 📚 Knowledge: ищет в собственной базе\n"
    "• 🎯 Planning: автоматически строит план\n"
    "• 🔍 Tools: DuckDuckGo",
    title="Advanced Agent",
    border_style="blue"
))


TypeError: Agent.__init__() got an unexpected keyword argument 'enable_agentic_context'. Did you mean 'enable_agentic_state'?

In [ ]:
# Утилиты для работы с продвинутым агентом

def quick_advanced_search(query: str):
    """Быстрый запрос без интерактивного режима"""
    console.print(Panel(
        f"[cyan]Запрос:[/cyan] {query}",
        title="Advanced Agent",
        border_style="cyan"
    ))
    response = advanced_agent.run(query)
    console.print(Panel(
        Markdown(response.content),
        title="Результат",
        border_style="green"
    ))
    return response

# Пример использования
# quick_advanced_search("Какие тренды в data engineering в 2025 году?")
console.print("[yellow]Advanced Agent готов! Используйте quick_advanced_search()[/yellow]")


---
## 3. Multi-Agent System с ролями и Structured Output

Построим команду Planner → Searcher → Analyzer → Writer. Planner и Analyzer/Writer вернут строгие Pydantic-модели.


In [ ]:
# Structured Output модели
class SearchPlan(BaseModel):
    query: str = Field(..., description="Оригинальный запрос")
    search_queries: List[str] = Field(..., description="Список уточненных запросов")
    strategy: str = Field(..., description="Стратегия исследования")
    expected_sources: int = Field(default=3, description="Минимум источников")

class AnalysisResult(BaseModel):
    key_findings: List[str] = Field(..., description="Ключевые находки")
    summary: str = Field(..., description="Краткое резюме")
    confidence: float = Field(..., ge=0.0, le=1.0, description="Уверенность (0-1)")

class FinalReport(BaseModel):
    title: str = Field(...)
    content: str = Field(...)
    sources: List[str] = Field(...)
    confidence_score: float = Field(..., ge=0.0, le=1.0)

console.print("[green]✓ Structured Output модели определены![/green]")


In [ ]:
# Агенты мультисистемы
planner_agent = Agent(
    name="Planner",
    role="system",
    model=OpenAIChat(
        id="Qwen/Qwen3-Next-80B-A3B-Instruct",
        api_key=api_key,
        base_url=base_url
    ),
    system_message=(
        "Ты стратег поиска.\n"
        "- Анализируй запрос пользователя.\n"
        "- Предлагай пошаговый план и уточняющие поисковые запросы.\n"
        "- Учитывай разные аспекты темы."
    ),
    response_model=SearchPlan,
    markdown=True
)

searcher_agent = Agent(
    name="Searcher",
    role="system",
    model=OpenAIChat(
        id="Qwen/Qwen3-Next-80B-A3B-Instruct",
        api_key=api_key,
        base_url=base_url
    ),
    tools=[DuckDuckGoTools()],
    system_message=(
        "Ты интернет-поисковик.\n"
        "- Всегда используй DuckDuckGo.\n"
        "- Собирать актуальные факты и перечислять источники."
    ),
    markdown=True
)

analyzer_agent = Agent(
    name="Analyzer",
    role="system",
    model=OpenAIChat(
        id="Qwen/Qwen3-Next-80B-A3B-Instruct",
        api_key=api_key,
        base_url=base_url
    ),
    system_message=(
        "Ты аналитик.\n"
        "- Выделяй ключевые находки.\n"
        "- Указывай уровень уверенности."
    ),
    response_model=AnalysisResult,
    markdown=True
)

writer_agent = Agent(
    name="Writer",
    role="system",
    model=OpenAIChat(
        id="Qwen/Qwen3-Next-80B-A3B-Instruct",
        api_key=api_key,
        base_url=base_url
    ),
    system_message=(
        "Ты редактор.\n"
        "- Создавай финальные отчеты в Markdown.\n"
        "- Добавляй список источников и оценку достоверности."
    ),
    response_model=FinalReport,
    markdown=True
)

console.print(Panel(
    "[green]✓ Мультиагентная система создана![/green]\n\n"
    "Агенты:\n"
    "1. Planner — стратегия\n"
    "2. Searcher — поиск\n"
    "3. Analyzer — аналитика\n"
    "4. Writer — финальный отчет",
    title="Multi-Agent System",
    border_style="magenta"
))


In [ ]:
# Оркестратор мультиагентной системы
def run_multi_agent_search(query: str):
    console.print(Panel(
        f"[bold cyan]Запрос:[/bold cyan] {query}",
        title="🚀 Multi-Agent System",
        border_style="cyan"
    ))

    plan_response = planner_agent.run(
        f"Создай план поиска для запроса: {query}"
    )
    plan = plan_response.content if hasattr(plan_response, "content") else plan_response

    console.print(Panel(
        f"План готов!\nСтратегия: {plan.strategy}\nЗапросы: {', '.join(plan.search_queries)}",
        title="📋 Planner",
        border_style="blue"
    ))

    search_prompt = "\n".join(f"- {item}" for item in plan.search_queries)
    search_response = searcher_agent.run(
        f"Выполни поиск по запросам:\n{search_prompt}\n\nОригинальный запрос: {query}"
    )

    console.print(Panel(
        "[green]Поиск выполнен![/green]",
        title="🔍 Searcher",
        border_style="green"
    ))

    analysis_response = analyzer_agent.run(
        f"Проанализируй результаты:\n\n{search_response.content}"
    )
    analysis = analysis_response.content if hasattr(analysis_response, "content") else analysis_response

    console.print(Panel(
        f"Ключевых находок: {len(analysis.key_findings)}\nУверенность: {analysis.confidence:.0%}",
        title="📊 Analyzer",
        border_style="yellow"
    ))

    report_response = writer_agent.run(
        f"Создай финальный отчет по теме '{query}'\n\nРезультаты:\n{search_response.content}\n\nАнализ:\n{analysis.summary}"
    )
    report = report_response.content if hasattr(report_response, "content") else report_response

    console.print(Panel(
        Markdown(report.content),
        title=report.title,
        border_style="magenta"
    ))

    console.print(Panel(
        "\n".join(report.sources) + f"\n\nУверенность: {report.confidence_score:.0%}",
        title="📚 Источники",
        border_style="blue"
    ))

    return report

# Пример использования
# run_multi_agent_search("Какие ключевые инициативы в области зеленой энергетики в 2025 году?")
console.print("[yellow]Multi-Agent System готов! Используйте run_multi_agent_search()[/yellow]")


---
## Итоги

- **Workflow** показывает, как строить флоу на функциях `search_step` → `analyze_step`.
- **Advanced Agent** объединяет planning + memory + knowledge и использует DuckDuckGo как tool.
- **Multi-Agent System** делит роли (Planner, Searcher, Analyzer, Writer) и валидирует ответы через Pydantic.
- Все конфигурации берут ключи из `.env` через `settings.py`, как в `ask_model.ipynb`.

Перед запуском раскомментируйте нужные вызовы ниже каждой секции.
